In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from model.seq2seq import Encoder, Decoder, Seq2Seq
from utils.tokenizer import Tokenizer

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from model.seq2seq import Encoder, Decoder, Seq2Seq
from utils.tokenizer import Tokenizer

# Configurações de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer e Dados de Treinamento
tokenizer = Tokenizer()
tokenizer.build_vocab([
    "oi", "olá!", "tudo bem?", "estou bem, obrigado.",
    "qual seu nome?", "sou seu assistente."
])

pairs = [
    ("oi", "olá!"),
    ("tudo bem?", "estou bem, obrigado."),
    ("qual seu nome?", "sou seu assistente."),
]

def tensorize(pair):
    src = torch.tensor([tokenizer.word2idx['<sos>']] + tokenizer.encode(pair[0]) + [tokenizer.word2idx['<eos>']])
    trg = torch.tensor([tokenizer.word2idx['<sos>']] + tokenizer.encode(pair[1]) + [tokenizer.word2idx['<eos>']])
    return src, trg

data = [tensorize(p) for p in pairs]

# Criar o DataLoader
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

def collate(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=0)
    trg_batch = pad_sequence(trg_batch, batch_first=True, padding_value=0)
    return src_batch, trg_batch

loader = DataLoader(ChatDataset(data), batch_size=2, collate_fn=collate)

# Definição do Modelo
INPUT_DIM = OUTPUT_DIM = len(tokenizer.word2idx)
EMB_DIM = 64
HID_DIM = 128

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM).to(device)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM).to(device)
model = Seq2Seq(enc, dec).to(device)

optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(ignore_index=0)

# Loop de treinamento
EPOCHS = 100
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for src, trg in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}: Loss = {epoch_loss:.4f}")

# Salvar o modelo treinado
torch.save(model.state_dict(), "model/checkpoints/seq2seq.pt")


In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from model.seq2seq import Encoder, Decoder, Seq2Seq
from utils.tokenizer import Tokenizer

# Configurações de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer e Dados de Treinamento
tokenizer = Tokenizer()
tokenizer.build_vocab([
    "oi", "olá!", "tudo bem?", "estou bem, obrigado.",
    "qual seu nome?", "sou seu assistente."
])

pairs = [
    ("oi", "olá!"),
    ("tudo bem?", "estou bem, obrigado."),
    ("qual seu nome?", "sou seu assistente."),
]

def tensorize(pair):
    src = torch.tensor([tokenizer.word2idx['<sos>']] + tokenizer.encode(pair[0]) + [tokenizer.word2idx['<eos>']])
    trg = torch.tensor([tokenizer.word2idx['<sos>']] + tokenizer.encode(pair[1]) + [tokenizer.word2idx['<eos>']])
    return src, trg

data = [tensorize(p) for p in pairs]

# Criar o DataLoader
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

def collate(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=0)
    trg_batch = pad_sequence(trg_batch, batch_first=True, padding_value=0)
    return src_batch, trg_batch

loader = DataLoader(ChatDataset(data), batch_size=2, collate_fn=collate)

# Definição do Modelo
INPUT_DIM = OUTPUT_DIM = len(tokenizer.word2idx)
EMB_DIM = 64
HID_DIM = 128

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM).to(device)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM).to(device)
model = Seq2Seq(enc, dec).to(device)

optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(ignore_index=0)

# Loop de treinamento
EPOCHS = 100
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for src, trg in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}: Loss = {epoch_loss:.4f}")

# Salvar o modelo treinado
torch.save(model.state_dict(), "model/checkpoints/seq2seq.pt")


In [ ]:
import cv2
from deepface import DeepFace

# Inicia a captura da webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detecta rostos e estima idade/gênero
    try:
        result = DeepFace.analyze(frame, actions=['age', 'gender'], enforce_detection=False)
        for face in result:
            x, y, w, h = face['region'].values()
            age = face['age']
            gender = face['dominant_gender']

            # Desenha o retângulo e as informações
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame, f"{gender}, {age}", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
    except Exception as e:
        print("Erro:", e)

    cv2.imshow("Webcam - Age & Gender", frame)

    # Tecla 'q' para sair
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
